# Coarse alignment with RANSAC
We are going to compute sparse correspondences using off-the-shelf deep feature detector and RANSAC to estimate a homography. 

In [ ]:
# TODO imports

Path and parameters

In [ ]:
n_scales = 7
iterations = 10000
tolerance = 0.05

min_size = 400

image_net = True # select False for MOCO

scale_r = 1.2


### Without alignment

### Coarse alignment